In [2]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')


In [3]:
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'config.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

In [4]:
from rich import print
from rest_framework.serializers import ValidationError as DRFValidationError
from rest_framework import serializers
from django.forms.utils import ErrorDict, ErrorList
from django.core.exceptions import ValidationError as DjangoValidationError
from rest_framework.response import Response
from apps.core.exceptions import FieldValidationError

from pydantic import BaseModel

In [4]:
class PydanticChildModel(BaseModel):
    int_field: int
    float_field: float

class ParentPydanticModel(BaseModel):
    string_field: str
    list_field: list[PydanticChildModel]

In [5]:
# ParentPydanticModel(
#     string_field='test', 
#     list_field=[
#         {
#             'int_field': 1, 
#             'float_field': 1.0,
#         },
#         {
#             'int_field': 2,
#             # 'float_field': 2.0,
#         }
#     ]
# )

# DRF Serializers

In [6]:
class ChildSerializer(serializers.Serializer):
    int_field = serializers.IntegerField()
    float_field = serializers.FloatField()


class ParentSerializer(serializers.Serializer):
    string_field = serializers.CharField()
    list_field = ChildSerializer(many=True)

In [7]:
serializer = ParentSerializer(
    data={
        'string_field': 'test',
        'list_field': [
            {
                'int_field': 1,
                'float_field': 1.0,
            },
            {
                'int_field': 2,
                # 'float_field': 2.0,
            }
        ]
    }
)

serializer.is_valid()

serializer.errors

Response(serializer.errors, status=400).data

{'list_field': [{}, {'float_field': [ErrorDetail(string='This field is required.', code='required')]}]}

# Django ErrorDict And ErrorList

In [6]:
nested_error_dict = {
    'string_field': 'test',
    'list_field': [
        {
            'int_field': 1,
            'float_field': 1.0,
        },
        {
            'int_field': 2,
            # 'float_field': 2.0,
        }
    ],
    'dict_field': {
        'int_field': 1,
        'float_field': 1.0,
    }
}

In [23]:
error_dict = ErrorDict(nested_error_dict)
error_dict

{'string_field': 'test',
 'list_field': [{'int_field': 1, 'float_field': 1.0}, {'int_field': 2}],
 'dict_field': {'int_field': 1, 'float_field': 1.0}}

In [10]:
ErrorList([{'string_field':'value'}, {'string_field':'value'}])

[{'string_field': 'value'}, {'string_field': 'value'}]

In [11]:
DjangoValidationError([{'string_field':'value'}, {'string_field':'value'}])

ValidationError(['value', 'value'])

In [21]:
DjangoValidationError({'string_field':'value'}).error_dict

{'string_field': [ValidationError(['value'])]}

In [13]:
django_val_error_dict = DjangoValidationError({'string_field':'value'}).error_dict
django_val_error_dict

{'string_field': [ValidationError(['value'])]}

In [7]:
FieldValidationError(nested_error_dict)

FieldValidationError({'string_field': 'test', 'list_field': [{'int_field': 1, 'float_field': 1.0}, {'int_field': 2}], 'dict_field': {'int_field': 1, 'float_field': 1.0}}

In [17]:
dj_list_val_error = DjangoValidationError(['This field requires a string input.'])
dj_dict_val_error = DjangoValidationError({'string_field':'value'})

In [19]:
type(dj_dict_val_error.error_dict)

dict

In [9]:
print(f'{dj_list_val_error.messages = }')
print(f'{dj_dict_val_error.message_dict = }')

dj_list_val_error.messages = ['This field requires a string input.']

dj_dict_val_error.message_dict = {'string_field': ['value']}

In [15]:
DjangoValidationError(
    [
        {
    'string_field': 'test',
    'list_field': [
        {
            'int_field': 1,
            'float_field': 1.0,
        },
        {
            'int_field': 2,
            # 'float_field': 2.0,
        }
    ],
    'dict_field': {
        'int_field': 1,
        'float_field': 1.0,
    }
},
        {
    'string_field': 'test',
    'list_field': [
        {
            'int_field': 1,
            'float_field': 1.0,
        },
        {
            'int_field': 2,
            # 'float_field': 2.0,
        }
    ],
    'dict_field': {
        'int_field': 1,
        'float_field': 1.0,
    }
}
    ],
)

AttributeError: 'ValidationError' object has no attribute 'error_list'